In [ ]:
import pandas as pd
import processor as pr
import loader as load
import predictor as pred
import config

# ge_phylum_stage = load.loadGEOverlappingTCMA("phylum", includeStage=True)
# ge_genus_stage = load.loadGEOverlappingTCMA("genus", includeStage=True)
# aak_ge_stage = load.loadGEWithClinical(includeStage=True)
test = [[1, 4], [8, 9]]
actual = [[1, 2], [2, 3]]
predictions = {}
all_predicted_labels = [y for x in test for y in x]
all_actual_labels = [y for x in actual for y in x]
all_sampling_iterations = [i for i, x in enumerate(actual) for y in range(len(x))]
predictions["predicted"] = all_predicted_labels
predictions["actual"] = all_actual_labels
predictions["iteration"] = all_sampling_iterations

# current_prediction_index = 0
# for pred in test:
#     predictions[current_prediction_index] = 
print(all_predicted_labels)
print(all_sampling_iterations)
pd.DataFrame.from_dict(predictions, orient='columns')

In [ ]:
# aak_ge takes a while. chokes during feature selection COAD even with 5
for target in config.prediction_targets[1:]:
    data, files = load.loadAll(includeStage=(target=="stage"), sameSamples=True, skipGenes=False)
    
    for sampling in config.sampling[:]:

        for selection in config.selection_types[:]:
            # print(data[1:], files[1:])
            # pred.runExperiments(data[1:2], files[1:2], target=target, sampling=sampling, selection=selection)
            pred.runExperiments(data[:], files[:], target=target, sampling=sampling, selection=selection)

In [ ]:
data, files = load.loadAll(includeStage=True, sameSamples=True)

In [ ]:
pred.runExperiments(data[:], files[:], target="stage", sampling="random_sampling")

In [ ]:
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, average_precision_score, log_loss
import numpy as np

d = data[1]
d = load.attachStageStatus(d)
x, y = pr.splitData(d, target="stage", project="ESCA")
print(y)
model = LinearRegression()
# pred.runRandomSampling(x, y, model)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.4, stratify=y, random_state=42)

model.fit(x_train, y_train)
y_predicted = model.predict(x_test)
y_predicted_rounded = np.rint(y_predicted)
y_predicted_rounded
# y_predicted
y.dtype


In [ ]:
# test = [ 5.3, 5.2]
# new = np.rint(test)
# maximum = max(y_train)

# test = np.clip(y_train, 0, 10)
# test
# https://stackoverflow.com/questions/43991252/anyway-to-average-different-confusion-matrices
# https://stackoverflow.com/questions/64916272/how-to-display-mean-classification-report-and-confusion-matrix-in-case-of-kfold?rq=1
# https://medium.com/analytics-vidhya/generation-of-a-concatenated-confusion-matrix-in-cross-validation-912485c4a972
# https://towardsdatascience.com/how-to-plot-a-confusion-matrix-from-a-k-fold-cross-validation-b607317e9874
y_predicted_rounded
from sklearn.metrics import confusion_matrix
# cm1 = pd.crosstab(pd.Series(y_predicted_rounded), pd.Series(y_test), rownames=['Predicted'], colnames=['Actual'], margins=True)
# cm1
print(y_test.values, y_predicted_rounded)
confusion_matrix(y_test, y_predicted_rounded)

In [ ]:
# test = [ 5.3, 5.2]
# new = np.rint(test)
# maximum = max(y_train)

# test = np.clip(y_train, 0, 10)
# test

y_predicted_rounded
cm1 = pd.crosstab(pd.Series(y_predicted_rounded), pd.Series(y_test), rownames=['Predicted'], colnames=['Actual'], margins=True)
cm1

In [ ]:
model.coef_
# features_with_coefficients = pd.DataFrame(zip(x_train.columns, model.coef_))
features_with_coefficients = pd.DataFrame({"feature":x_train.columns,"coefficients":np.transpose(model.coef_)})
features_with_coefficients_abs = features_with_coefficients.copy()
features_with_coefficients_abs["coefficients"] = features_with_coefficients_abs.apply(lambda row: abs(row.coefficients), axis=1)
features_with_coefficients_abs
sorted_features = features_with_coefficients_abs.sort_values("coefficients", ascending=False)
top_features = sorted_features.head(20)["feature"].values
x_train[top_features]

best_indices = [x_train.columns.get_loc(c) for c in x_train.columns if c in top_features]


print(x_train[top_features].columns)

print(x_train.iloc[:, best_indices].columns)
linreg_ranked_features = pr.selectFeatures(x, y, 20, "linreg")
print(x_train.iloc[:, linreg_ranked_features].columns)
linreg_ranked_features[:20]